# 重要版本之一！！！——朴素贝叶斯分列模型

## 导入所需要的库
在pycharm中将光标（即“|”）移至需要导入的库上，按ALT+Enter，再选择“install package *”,稍等片刻即可

In [18]:
import jieba
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

### 提示输出格式规范

In [2]:
def format_output(_str):
    print(">>> " + _str)
    return None

## 导入数据集
在jupyter notebook上发现，数据集过大会有一些警告，可能是由于他们服务器的原因。这里使用train1.tsv(含有3865条数据)作为模型测试，你们调试的时候可以根据自己的选择。

In [3]:
# 导入数据集
format_output("正在导入数据...")
train_tsv = "./data/train1.tsv"
# test_tsv = "./data.test.tsv"
train_df = pd.read_csv(train_tsv,sep='\t')
# test_df = pd.read_csv(test_tsv,sep='\t')
format_output("加载数据集已成功！")

>>> 正在导入数据...
>>> 加载数据集已成功！


In [4]:
# 初步分列
item_name = train_df['ITEM_NAME']
tri_type = train_df['TYPE']

In [5]:
train_df.head(10)

## 根据“生物分类学”中的“目（ORDER）、科（FAMILY）、属（SPECIES）”进行多级分列
设想：能不能设计使用一种类似于“漏斗形”的筛选分类器，筛选出大类，并继续进行细分。
关于本次实验：先采用大类进行模拟预测，验证是否可行！

In [6]:
format_output("正在基于‘TYPE’列创建order-fimaly-species分列，请稍后...")
# 将原始分列标签按照order-fimaly-species分列，并保存数据
temp_df = train_df['TYPE'][:].str.split('--',expand=True)
train_ndf = train_df.drop('TYPE',axis=1).join(temp_df)
train_ndf.rename(columns={0:'ORDER',1:'FAMILY',2:'SPECIES'},inplace=True)
format_output("创建分列成功！")
train_ndf.to_csv("./data/tri_type.csv",index=False,encoding="utf_8_sig")
# train_ndf.info()
format_output("调整分列后的数据集已导出！")

>>> 正在基于‘TYPE’列创建order-fimaly-species分列，请稍后...
>>> 创建分列成功！
>>> 调整分列后的数据集已导出！


In [7]:
# 单独抽出分列
order_ = train_ndf['ORDER']
fimaly_ = train_ndf['FAMILY']
species_ = train_ndf['SPECIES']
# print(order_)

In [8]:
# 各列、各类别统计数目
order_counts = order_.value_counts()
fimaly_counts = fimaly_.value_counts()
species_counts = species_.value_counts()
# print(order_counts)

## 降噪处理
由于样本含有过多数字、标点符号、特殊字符、英文（由于大多表示商品编号、单位等，因此在此处不作为分类参考）

In [10]:
# 消除噪点，即数字、标点符号、特殊字符等非语义类文本
from string import digits,ascii_letters,punctuation
# printable = digits + ascii_letters + punctuation + whitespace

def remove_noise(_str):
    noise = digits + ascii_letters + punctuation + "★【】！（）：；“”‘’《》，。、？"
    return _str.translate(str.maketrans('','',noise))

## jieba分词
切分原理：jieba分词依据内置或者用户自定义的中文语料库进行字段的切分
不足：语料库中可能未收集到新的词汇

In [11]:
# jieba分词函数
def cutword(dataSet):
    format_output("正在切分字段，这可能需要一段时间！")
    dataSetcw = [""] * len(dataSet)
    cw = lambda x: list(jieba.cut(x,cut_all=False))
    for i in range(len(dataSet)):
        dataSetcw[i] = ' '.join(cw(dataSet.iloc[i]))
        dataSetcw[i] = remove_noise(dataSetcw[i])
    format_output("切分完成！")
    return dataSetcw

In [12]:
print(cutword(ttest['ITEM_NAME'])[1])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\LOUIS_~1\AppData\Local\Temp\jieba.cache


>>> 正在切分字段，这可能需要一段时间！


Loading model cost 1.602 seconds.
Prefix dict has been built succesfully.


>>> 切分完成！
腾讯  币 充值  元  币  币 自动 充值


In [13]:
item_name_cw = cutword(item_name)

>>> 正在切分字段，这可能需要一段时间！
>>> 切分完成！


In [17]:
# 将文本中的词语转换为词频矩阵
format_output("正在统计词条，请稍后...")
cv = CountVectorizer(min_df=1)
# 计算各个词语出现的次数
cv_fit = cv.fit_transform(item_name_cw[:])

# set of words（SOW） 词集模型 - 获取词袋中所有文本关键词
print("打印所有的特征名称")
print(cv.get_feature_names())

# bag of words（BOW） 词袋模型
print("打印整个文本矩")
print(cv_fit.toarray())
temp = cv_fit.toarray()
# print("矩阵大小：",temp.shape)
print("打印所有的列相加(统计特征名称出现次数)")
print(cv_fit.toarray().sum(axis=0))
print("打印所有的行相加")
print(cv_fit.toarray().sum(axis=1))

打印所有的特征名称
['一业', '一个', '一个月', '一件', '一只', '一号', '一品', '一图', '一壶', '一夜', '一套', '一对', '一帆风顺', '一条', '一款', '一物', '一生', '一百', '一石', '一线', '一见倾心', '一起', '一路', '一轮', '一钱', '一飞冲天', '一鸣惊人', '一鹿', '丁香', '七夕', '七度', '七果果', '七窍玲珑', '七采', '万事', '万事如意', '万生', '万福', '三个', '三件套', '三元', '三凤牌', '三双', '三叶草', '三号', '三合', '三圈', '三彩', '三彩春', '三彩玉', '三彩玉珠', '三支', '三星', '三朵', '三环', '三生', '三眼', '三缘', '三脚', '三色', '三角', '三足', '三轮', '上古', '上品', '上尊', '上山', '上市', '上帝', '下单', '不动', '不动尊', '不动明', '不同', '不含', '不带', '不干胶', '不弃', '不求人', '不离', '不规则', '不锈钢', '不顺', '丑牛', '专业', '专享', '专利', '专卖店', '专属', '专柜', '专用', '世奇', '世奇笔', '世家', '世琦', '世瑞宝', '世界', '丘比特', '丘玉', '丙申', '东圣', '东方', '东陵', '丝娜', '丝烧', '丝般', '丢失', '两件', '两双', '两戴', '两条', '两用', '两色', '两面', '严咒', '个人版', '个性', '中低', '中医', '中华', '中号', '中国', '中国娃娃', '中大模', '中孚', '中性', '中棉', '中款', '中版', '中版约', '中秋', '中融', '中袜', '中邮', '丰泽', '串珠', '为准', '主义', '丽影', '丽都', '义薄云天', '之地', '之心', '之恋', '之星', '之石', '之翼', '之花', '之间', '乌木', '乌金', '乌银', '乐天', '乐灵', '乖乖', '九个', '九壹', '九天', '九洲

[1 2 2 ... 2 1 1]
打印所有的行相加
[10  4  3 ... 10 12 11]


In [20]:
# 训练集和测试集的科学比例拆分
x_train,x_test,y_train,y_test = train_test_split(item_name_cw,order_,test_size=0.25)

In [22]:
# 对数据进行特征抽取
tf = TfidfVectorizer()

# 以训练集中的词的列表进行每个商品名称的重要性统计
x_tra = tf.fit_transform(x_train)
print(tf.get_feature_names())
x_tes = tf.transform(x_test)

# 进行朴素贝叶斯算法的预测
mlt = MultinomialNB(alpha=1.0)
print(x_tra)
mlt.fit(x_tra, y_train)

y_predict = mlt.predict(x_tes)
print("预测的文章类别为：", y_predict)

# 得出准确率
print("准确率为：", mlt.score(x_tes, y_test))

['一业', '一个', '一个月', '一件', '一只', '一号', '一品', '一图', '一夜', '一套', '一对', '一帆风顺', '一条', '一款', '一物', '一生', '一百', '一石', '一线', '一见倾心', '一起', '一路', '一钱', '一鸣惊人', '一鹿', '丁香', '七夕', '七度', '七果果', '七窍玲珑', '万事如意', '万福', '三个', '三件套', '三元', '三凤牌', '三双', '三合', '三圈', '三彩', '三彩春', '三彩玉珠', '三星', '三朵', '三环', '三生', '三缘', '三色', '三角', '三足', '三轮', '上古', '上品', '上尊', '上山', '上市', '下单', '不动', '不动尊', '不动明', '不同', '不含', '不带', '不干胶', '不弃', '不离', '不规则', '不锈钢', '不顺', '专享', '专利', '专卖店', '专属', '专柜', '专用', '世奇', '世奇笔', '世家', '世琦', '世瑞宝', '世界', '丘玉', '丙申', '东圣', '东方', '东陵', '丝娜', '丝烧', '丝般', '丢失', '两戴', '两条', '两用', '两色', '两面', '个人版', '个性', '中低', '中医', '中华', '中号', '中国', '中国娃娃', '中孚', '中性', '中棉', '中款', '中秋', '中融', '中袜', '丰泽', '串珠', '为准', '主义', '丽影', '丽都', '义薄云天', '之地', '之心', '之恋', '之星', '之翼', '之花', '之间', '乌木', '乌金', '乌银', '乐天', '乐灵', '乖乖', '九壹', '九天', '九洲', '九牧', '乡村', '书包', '书架', '书柜', '书页', '乱步', '乳白', '事业有成', '事事', '二凤', '二十', '二层', '二款', '二胎', '二轮', '二选', '云上', '五号', '五彩', '五行', '五角星', '五谷', '五谷丰登', '五谷杂粮', '五金', '交叉', '亥